## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-24-14-49-25 +0000,nyt,Xi’s Purge of China’s Military Brings Its Top ...,https://www.nytimes.com/2026/01/24/world/asia/...
1,2026-01-24-14-44-00 +0000,wsj,Trump Threatens New Tariffs on Canada Over China,https://www.wsj.com/politics/trump-threatens-n...
2,2026-01-24-14-43-25 +0000,nyt,Why the Power May Not Go Out Until After the S...,https://www.nytimes.com/2026/01/24/business/en...
3,2026-01-24-14-36-53 +0000,nypost,NYC dogs recreate celeb looks from the Golden ...,https://nypost.com/2026/01/24/entertainment/ny...
4,2026-01-24-14-33-43 +0000,nypost,"Where do the most liars, cheats and fraudsters...",https://nypost.com/2026/01/24/us-news/nevada-m...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2433/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,trump,55
323,ice,17
155,greenland,14
2,china,12
146,russia,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
269,2026-01-23-16-09-21 +0000,nyt,Trump Withdraws Carney’s Invitation for Canada...,https://www.nytimes.com/2026/01/23/us/politics...,96
21,2026-01-24-13-34-20 +0000,nypost,Trump tells The Post he has seized Venezuela o...,https://nypost.com/2026/01/24/us-news/trump-te...,96
26,2026-01-24-13-19-05 +0000,nyt,"Despite Trump’s Words, China and Russia Are No...",https://www.nytimes.com/2026/01/24/us/politics...,94
10,2026-01-24-14-21-45 +0000,cbc,Trump threatens 100% tariffs on all Canadian g...,https://www.cbc.ca/news/politics/trump-tariff-...,94
1,2026-01-24-14-44-00 +0000,wsj,Trump Threatens New Tariffs on Canada Over China,https://www.wsj.com/politics/trump-threatens-n...,93


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
269,96,2026-01-23-16-09-21 +0000,nyt,Trump Withdraws Carney’s Invitation for Canada...,https://www.nytimes.com/2026/01/23/us/politics...
242,53,2026-01-23-18-10-10 +0000,nypost,DHS officials debunk viral claims ICE used 5-y...,https://nypost.com/2026/01/23/us-news/dhs-offi...
67,46,2026-01-24-09-08-00 +0000,wsj,China’s Xi Places His Top General Under Invest...,https://www.wsj.com/world/china/chinas-xi-plac...
199,46,2026-01-23-20-39-22 +0000,nypost,"US, Ukraine and Russia hold first-ever trilate...",https://nypost.com/2026/01/23/world-news/us-uk...
21,41,2026-01-24-13-34-20 +0000,nypost,Trump tells The Post he has seized Venezuela o...,https://nypost.com/2026/01/24/us-news/trump-te...
9,41,2026-01-24-14-24-51 +0000,nypost,Winter Storm Fern could drop a foot of snow on...,https://nypost.com/2026/01/24/us-news/nyc-wake...
84,33,2026-01-24-05-31-24 +0000,startribune,"The latest: Thousands march, scores of Minneso...",https://www.startribune.com/ice-raids-minnesot...
116,31,2026-01-24-01-11-10 +0000,nyt,"Ryan Wedding, Canadian Ex-Snowboarder Accused ...",https://www.nytimes.com/2026/01/23/world/canad...
290,30,2026-01-23-14-59-43 +0000,bbc,Trump 'wrong' to claim Nato troops avoided Afg...,https://www.bbc.com/news/articles/czr444j671vo...
55,28,2026-01-24-10-50-00 +0000,startribune,NFL championship game picks: NFL’s final four ...,https://www.startribune.com/nfl-championship-g...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
